In [44]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
import pandas as pd
from scipy import signal
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
#import seaborn as sns
"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

In [45]:
print(torch.cuda.get_device_name())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GeForce RTX 2060 SUPER


In [46]:
from glob import glob
import pickle
import numpy as np

### Change to requried path to access data locally, too big too push all data into github
#ROOT_PATH = "C:/Users/Administrator/cse151b-spring2022/argo2/"
ROOT_PATH = "D:/School/cse151B/argo2/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

        return torch.from_numpy(inputs).float(), torch.from_numpy(outputs).long()

    if split=="test":
    
        return torch.from_numpy(inputs).float(), torch.from_numpy(np.array([]))

    

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None, device='cpu'):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform
        self.split = split
        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)
        self.inputs = self.inputs.to(device)
        self.outputs = self.outputs.to(device)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        if self.split == 'train':
            data = (self.inputs[idx], self.outputs[idx])
        if self.split == 'test':
            data = self.inputs[idx]
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
#train_dataset  = ArgoverseDataset(city = city, split = split, device=device)

###  Data Processing

In [47]:
# wrapper function to select proportion of a cities random examples w/o replacement from each city and put all data into one list
# purpose: whole dataset is too big and might be redundant
def randomCitySampler(prop,scalingFactor):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    samples = []
    for c in cities:
        # get city data
        temp_dataset = ArgoverseDataset(city = c, split = "train", device=device)

        numProp = int(len(temp_dataset) * prop)

        # get N number of random indicies
        ind = random.sample(range(0, len(temp_dataset)), numProp)
        #print(ind)
        # push all data indicies into samples list
        for i in ind: 
            x = temp_dataset[i][0]/scalingFactor
            y = temp_dataset[i][1]/scalingFactor
            samples.append((x,y))
            
    return samples


In [48]:
### constants for generating Dataset
proportionOfEntireData = 0.01
seqLen = 50
stepSize = 5
batch_sz = 64  # batch size 
scaling_factor = 1000 # divides each value by this number to scale it

In [49]:
# create train dataset, with proportion to actual amount data
sampleTest = randomCitySampler(proportionOfEntireData,scaling_factor)
len(sampleTest)

2035

In [50]:
# generate sequences of length seqLength and specific step size
def sequenceGenerator(data, seqLen=40, stepSize=5):
    newData = []
    for d in data:
        # concat X and Y together
        temp = torch.cat([d[0],d[1]])
        # make X of length SeqLen and Y is next x,y coordinate pair
        for i in range(0,len(temp)-seqLen, stepSize):
            x = temp[i:i + seqLen]
            #flatX = Variable(torch.tensor([item for sublist in x for item in sublist])).to(device)
            flatX = torch.flatten(Variable(torch.tensor(x)).to(device))
            y = temp[i+seqLen]
            newData.append((flatX,y))
            
        
    return newData

In [51]:
# generate sequences
train_seq_data = sequenceGenerator(sampleTest,seqLen, stepSize)
len(train_seq_data)

<ipython-input-50-8e56f7b34fb3>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  flatX = torch.flatten(Variable(torch.tensor(x)).to(device))


24420

In [52]:
# should be a vector of size (2*seqLen,2) for each example
train_seq_data[0], len(train_seq_data[0][0])

((tensor([ 1.5579, -1.8898,  1.5581, -1.8894,  1.5583, -1.8888,  1.5585, -1.8882,
           1.5588, -1.8875,  1.5591, -1.8866,  1.5594, -1.8857,  1.5598, -1.8848,
           1.5602, -1.8838,  1.5606, -1.8828,  1.5609, -1.8819,  1.5613, -1.8810,
           1.5617, -1.8801,  1.5620, -1.8791,  1.5624, -1.8782,  1.5628, -1.8773,
           1.5631, -1.8764,  1.5635, -1.8755,  1.5638, -1.8746,  1.5642, -1.8737,
           1.5645, -1.8727,  1.5649, -1.8718,  1.5652, -1.8709,  1.5656, -1.8700,
           1.5659, -1.8691,  1.5663, -1.8682,  1.5666, -1.8673,  1.5670, -1.8664,
           1.5673, -1.8655,  1.5677, -1.8646,  1.5680, -1.8638,  1.5684, -1.8629,
           1.5687, -1.8620,  1.5690, -1.8612,  1.5694, -1.8603,  1.5697, -1.8595,
           1.5700, -1.8586,  1.5704, -1.8578,  1.5707, -1.8570,  1.5710, -1.8561,
           1.5713, -1.8553,  1.5716, -1.8546,  1.5719, -1.8538,  1.5722, -1.8530,
           1.5725, -1.8523,  1.5728, -1.8515,  1.5731, -1.8508,  1.5734, -1.8501,
           1.573

In [53]:
# create loader
train_loader = DataLoader(train_seq_data,batch_size=batch_sz)

In [54]:
# check shape is correct
train_features, train_labels = next(iter(train_loader))
len(train_features[0]), train_labels[0]
# shape is correct

(100, tensor([ 1.5740, -1.8480], device='cuda:0'))

### Linear Regression

In [55]:
# model parameters
#num_epochs = 60
learning_rate = 0.0001

input_size = seqLen*2 #number of features
hidden_size = 128 #number of features in hidden state
num_layers = 2 #number of stacked lstm layers

output_size = 2 #number of output classes 


In [56]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize, hidden_size):
        super(linearRegression, self).__init__()
        self.linear1 = torch.nn.Linear(inputSize, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, outputSize)

    def forward(self, x):
        out = self.linear1(x)
        out = self.linear2(out)
        return out

In [57]:
lr = linearRegression(input_size, output_size, hidden_size)
lr = lr.to(device)
lr

linearRegression(
  (linear1): Linear(in_features=100, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=2, bias=True)
)

In [58]:
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(lr.parameters(), lr=learning_rate)


### Training

In [59]:
epochs = 200

for i in range(epochs):
    for seq, labels in train_loader:
        lr.train()
        seq = seq.to(device)
        labels = labels.to(device)

        

        optimizer.zero_grad()
        #lr.hidden_cell = (torch.zeros(num_layers, 1, lr.hidden_layer_size).to(device),torch.zeros(num_layers, 1, lr.hidden_layer_size).to(device))
        
        
        y_pred = lr(seq)
        
        #print(seq.shape)
        #print(y_pred.shape, labels.shape)
        #break

        loss = loss_function(y_pred, labels)
        loss.backward()
        optimizer.step()

    if i%10 == 0:
        print(f'epoch: {i:3} loss: {loss.item():10.8f}')

    if i%50 == 0:
        modelPath = "lrTorchEpoch{0}.pt".format(i)
        torch.save(lr.state_dict(), modelPath)

print(f'epoch: {i:3} loss: {loss.item():10.10f}')

epoch:   0 loss: 0.01738375
epoch:  10 loss: 0.00339728
epoch:  20 loss: 0.00181302
epoch:  30 loss: 0.00101192
epoch:  40 loss: 0.00060252
epoch:  50 loss: 0.00039042
epoch:  60 loss: 0.00027862
epoch:  70 loss: 0.00021836
epoch:  80 loss: 0.00018503
epoch:  90 loss: 0.00016602
epoch: 100 loss: 0.00015481
epoch: 110 loss: 0.00014797
epoch: 120 loss: 0.00014364
epoch: 130 loss: 0.00014080
epoch: 140 loss: 0.00013887
epoch: 150 loss: 0.00013751
epoch: 160 loss: 0.00013652
epoch: 170 loss: 0.00013577
epoch: 180 loss: 0.00013518
epoch: 190 loss: 0.00013470
epoch: 199 loss: 0.0001343318


### Prediction

In [60]:
# need to cycle for each test set and use LSTM to predict 
# add results for each city to dataframe 
# concatinate all dataframes

def validation(model):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc","palo-alto"]

    # all the data frames
    allDF = []
    with torch.no_grad():
        for c in cities:
            
            test_dataset = ArgoverseDataset(city=c, split='test', device=device)
            test_loader = DataLoader(test_dataset,batch_size=128)

            cityPredictions = []
            for t in test_loader.dataset:
                model.eval()
                flat = torch.flatten(t)
                currentPred = []

                for i in range(60):
                    #print(flat)
                    #print(len(flat)-seqLen*2)

                    pred = torch.flatten(model(flat[len(flat)-seqLen*2:].view(1,seqLen*2)))

                    #print(flat[len(flat)-seqLen*2:].view(1,80))
                    #print(pred)

                    currentPred.append(pred)

                    #print(torch.flatten(pred).shape)
                    #print(flat.shape)
                    
                    flat = torch.cat((flat,pred),0)
                    #print(flat)

                #print(len(flat[100:]))
                cityPredictions.append(flat[100:].detach().to('cpu').numpy()*scaling_factor)
        
            df = pd.DataFrame(cityPredictions)
            df.columns = ['v' + str(i) for i in (range(120))]
            df['ID'] = [str(i) + '_' + c for i in (range(len(test_loader.dataset)))]
            allDF.append(df)
            
    return allDF


In [61]:
tempValDF = validation(lr)


In [62]:
len(tempValDF)

6

In [63]:
lrPredFinal = pd.concat(tempValDF)
lrPredFinal.ID

0             0_austin
1             1_austin
2             2_austin
3             3_austin
4             4_austin
             ...      
1681    1681_palo-alto
1682    1682_palo-alto
1683    1683_palo-alto
1684    1684_palo-alto
1685    1685_palo-alto
Name: ID, Length: 29843, dtype: object

In [64]:
lrPredFinal.to_csv("lrtorch.csv", index=False)

In [65]:
torch.save(lr.state_dict(), "lrtorchfinal.pt")

In [66]:
lrPredFinal.head()

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v111,v112,v113,v114,v115,v116,v117,v118,v119,ID
0,8.640423e+03,-5.746916e+05,8.036583e+03,-5.739679e+05,7.921162e+03,-5.730536e+05,8.366955e+03,-5.727774e+05,8.532963e+03,-5.737065e+05,...,-5.719129e+05,1.906464e+03,-5.719819e+05,2.837086e+03,-5.718892e+05,2.998597e+03,-5.724570e+05,2.732193e+03,-5.724031e+05,0_austin
1,-3.471135e+05,2.434854e+04,-3.473527e+05,2.380774e+04,-3.473492e+05,2.371947e+04,-3.475309e+05,2.334900e+04,-3.483431e+05,2.355387e+04,...,2.121599e+04,-3.486686e+05,2.144767e+04,-3.489024e+05,2.134151e+04,-3.487441e+05,2.159085e+04,-3.488446e+05,2.129664e+04,1_austin
2,5.093472e+04,-2.475840e+05,5.093151e+04,-2.475861e+05,5.092859e+04,-2.475858e+05,5.092356e+04,-2.475859e+05,5.091556e+04,-2.475804e+05,...,-2.474736e+05,5.077775e+04,-2.474697e+05,5.077187e+04,-2.474680e+05,5.076967e+04,-2.474632e+05,5.076676e+04,-2.474626e+05,2_austin
3,-1.137234e+05,1.795034e+06,-1.136632e+05,1.795161e+06,-1.136764e+05,1.795153e+06,-1.136526e+05,1.795231e+06,-1.134506e+05,1.795178e+06,...,1.794926e+06,-1.122399e+05,1.794846e+06,-1.121771e+05,1.794854e+06,-1.122089e+05,1.794780e+06,-1.121609e+05,1.794844e+06,3_austin
4,1.167985e+06,-6.255224e+05,1.168758e+06,-6.265594e+05,1.168962e+06,-6.277429e+05,1.168398e+06,-6.281635e+05,1.168051e+06,-6.268896e+05,...,-6.285362e+05,1.176362e+06,-6.283761e+05,1.175101e+06,-6.284988e+05,1.174925e+06,-6.276844e+05,1.175249e+06,-6.277981e+05,4_austin
